In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.interactive.interactive_beam import * 

bp = beam.Pipeline(interactive_runner.InteractiveRunner())

init_pcoll = bp |  'Init Create' >> beam.Create(range(1000))
squares = init_pcoll | 'Square' >> beam.Map(lambda x: x*x)
cubes = init_pcoll | 'Cube' >> beam.Map(lambda x: x**3)

In [ ]:
show(cubes)

In [ ]:
class Foo(object):
    def __init__(self, x, y, z):
        self.init = x
        self.square = y
        self.cube = z
        self.nested_bar = NestedBar(x,y)
        self.iterable_bar = IterableBar(x)

class NestedBar(object):
    def __init__(self, a, b):
        self.val = a + b

class IterableBar(object):
    def __init__(self, a):
        self.val = [a] * 10
        
exp = init_pcoll | beam.Map(lambda x: Foo(x, x*x, x**3))

In [ ]:
show(exp)

In [ ]:
bp.run()

In [ ]:
# As long as it's watched, a pipeline defined in a function can also have interactivity augmented.
def build_pipeline():
    bp = beam.Pipeline(interactive_runner.InteractiveRunner())
    init_pcoll = bp |  'Init Create' >> beam.Create(range(1000))
    exp = init_pcoll | 'Exp' >> beam.Map(lambda x: Foo(x, x*x, x**3))
    watch(locals())
    return bp, init_pcoll, exp

_, _, local_exp = build_pipeline()
show(local_exp)

In [ ]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import StandardOptions

pipeline_options = PipelineOptions()
pipeline_options.view_as(StandardOptions).streaming = True
sp = beam.Pipeline(interactive_runner.InteractiveRunner(),
                  options=pipeline_options)

taxi_rides = sp | "Read" >> beam.io.ReadFromPubSub(subscription='projects/ningk-test-project/subscriptions/test')

In [ ]:
# Once streaming cache and test stream are avaialble, enable the streaming pipeline run and visualization.
# sp_result = sp.run()
# visualize(taxi_rides)

# import time
# time.sleep(10)
# sp_result.cancel()

In [ ]:
# Even if there are multiple pipelines defined in a single notebook, the result should still be accessible correctly from bp.
visualize(exp)